In [28]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge # Ridge algorithm
from sklearn.linear_model import Lasso # Lasso algorithm
from sklearn.linear_model import BayesianRidge # Bayesian algorithm
from sklearn.linear_model import ElasticNet # ElasticNet algorithm

In [29]:
df1 = pd.read_json('D:\Data Zaky\Semester 5\Tugas Besar\Web Scraping\scrapeapp\data\data_property.json')
df1.head()

,id,kota,kamar_tidur,kamar_mandi,car_port,luas_tanah,luas_bangunan,harga
0,1,Tangerang,2,1,1,55,27,399000000
1,2,Tangerang,6,2,1,91,170,1320000000
2,3,Tangerang Selatan,3,2,1,76,67,1680000000
3,4,Tangerang,2,1,1,119,62,960000000
4,5,Tangerang,3,2,1,144,180,1650000000


In [30]:
df1 = df1.drop(columns=["id"])
df1.head()

,kota,kamar_tidur,kamar_mandi,car_port,luas_tanah,luas_bangunan,harga
0,Tangerang,2,1,1,55,27,399000000
1,Tangerang,6,2,1,91,170,1320000000
2,Tangerang Selatan,3,2,1,76,67,1680000000
3,Tangerang,2,1,1,119,62,960000000
4,Tangerang,3,2,1,144,180,1650000000


In [31]:
dummy_kota = pd.get_dummies(df1.kota)
df1 = df1.drop(columns=["kota"])

In [32]:
train_df = pd.concat([df1, dummy_kota], axis='columns')
train_df.head()

,kamar_tidur,kamar_mandi,car_port,luas_tanah,luas_bangunan,harga,Badung,Bandung,Bekasi,Bogor,...,Palembang,Semarang,Serang,Sidoarjo,Sleman,Solo,Surabaya,Tangerang,Tangerang Selatan,Yogyakarta
0,2,1,1,55,27,399000000,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,6,2,1,91,170,1320000000,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,2,1,76,67,1680000000,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2,1,1,119,62,960000000,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,3,2,1,144,180,1650000000,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [33]:
X = train_df.drop(columns=["harga"])
y = np.log(train_df['harga'])

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [45]:
ridge = Ridge(alpha = 5)
ridge.fit(X_train, y_train)
ridge.score(X_test, y_test)

0.4221612224502145

In [46]:
def predict_price(kota, kamar_tidur, kamar_mandi, car_port, luas_tanah, luas_bangunan): 
    kota_index = np.where(X.columns==kota.title())[0][0]
    x = np.zeros(len(X.columns))
    x[0] = kamar_tidur
    x[1] = kamar_mandi
    x[2] = car_port
    x[3] = luas_tanah
    x[4] = luas_bangunan
    
    if kota_index >= 0:
        x[kota_index] = 1
    return round(np.exp(ridge.predict([x])[0]))

In [47]:
predict_price("Tangerang Selatan", 3, 3, 1, 250, 195)

2040610841

In [48]:
import pickle
with open('../api/models/house_price_prediction.pickle','wb') as f:
    pickle.dump(ridge, f)

In [50]:
import json
columns = {
    'data_columns' : [col for col in X.columns]
}

with open('../api/models/house_columns.json', 'w') as f:
    f.write(json.dumps(columns))
